In [33]:
import folium
from folium.plugins import HeatMapWithTime
import demand_forecasting as df
import numpy as np
import os
import config

# Assuming df.demand_forecasting() returns the provided 9x15 matrix (9 bus stops, 15 hours)
demand_forecast = df.demand_forecasting()

# Convert to numpy array for easier manipulation
data_np = np.array(demand_forecast)

# Find min and max values in the array
min_val = data_np.min()
max_val = data_np.max()

# Normalize the data, round to 4sf
normalized_data = np.round((data_np - min_val) / (max_val - min_val), 4)

# Convert back to a list if needed
normalized_data_list = normalized_data.tolist()
print(normalized_data_list)

# Initialize map with a standard theme (OpenStreetMap as a fallback)
m = folium.Map(
    location=config.NUS_COORDINATES, 
    zoom_start=15, 
    control_scale=True,
    tiles="Cartodb dark_matter"
)

# Coordinates for each bus stop with names (sorted by alphabetical order of bus stop name)
bus_stops = sorted(config.BUS_STOP_COORDINATES.items(), key=lambda pair: pair[0])

# Adjusting specific labels position
adjusted_bus_stops = {
    "Kent Ridge MRT / Opp Kent Ridge MRT": (0, 25),  
    "UHC / Opp UHC": (0, 15),  
    "LT27 / S17": (0, 15), 
    "BIZ2 / Opp HSSML": (0, 10),  
    "LT13 / Ventus": (0, 10),  
    "UTown": (0, 15),
    "COM3": (0, 10),
    "PGP": (0, 15),
    "IT / CLB": (0, 15)
}

# Add bus stop names as DivIcons (simple HTML text)
for name, coords in bus_stops:
    # Check if the bus stop has an adjusted position
    offset = adjusted_bus_stops.get(name, (0, 0))

    folium.Marker(
        location=coords,
        icon=folium.DivIcon(
            icon_size=(90, 36),  # Adjust size to fit the text
            icon_anchor=(45, 18 + offset[1]),  # Adjust position based on the offset (vertical shift)
            html=f'<div style="font-size: 7px; font-weight: bold; color: black; background: #ffffff; padding: 2px 5px; border-radius: 5px; text-align: center;">{name}</div>'
        )
    ).add_to(m)

# Prepare data for HeatMapWithTime 
heat_data = []
for hour in range(15):  # 15 hours
    hour_data = []
    for stop_index, (name, coords) in enumerate(bus_stops):
        demand = normalized_data_list[stop_index][hour]  # Get demand at this stop and hour

        # Add this data point to the heat_data for visualization in each hourly interval
        hour_data.append([coords[0], coords[1], demand])

    # Append hour data to heat_data
    heat_data.append(hour_data)

# Add HeatMapWithTime to animate demand over time
HeatMapWithTime(
    data=heat_data, 
    radius=20, 
    auto_play=True,
    max_opacity=0.8
).add_to(m)

# Save the map as an HTML file
m.save(os.path.join(os.path.abspath(""),
            f"../visualisations/timelapses/demand_heatmap.html"))

Model accuracy before feature selection, for training data: 0.9260834455490112
Model accuracy before feature selection: -0.1266927719116211
Model accuracy after feature selection, for training data: 0.682835578918457
Model accuracy after feature selection: -0.0874948501586914
[[20, 17, 15, 36, 6, 24, 28, 16, 21, 23, 19, 13, 26, 18, 21], [44, 13, 21, 27, 24, 13, 10, 32, 20, 21, 12, 20, 20, 18, 9], [19, 13, 28, 21, 9, 17, 25, 25, 23, 25, 22, 12, 17, 14, 9], [46, 23, 22, 31, 32, 40, 15, 25, 39, 7, 22, 31, 24, 22, 0], [29, 23, 33, 10, 14, 34, 20, 25, 0, 5, 0, 29, 8, 0, 0], [16, 11, 19, 29, 14, 29, 28, 13, 17, 10, 19, 7, 6, 7, 0], [8, 39, 26, 21, 7, 8, 27, 24, 8, 32, 23, 9, 14, 19, 51], [0, 30, 21, 21, 9, 14, 35, 42, 32, 11, 10, 15, 0, 0, 17], [0, 33, 0, 22, 20, 23, 11, 11, 20, 18, 21, 26, 11, 0, 0]]
[[0.3922, 0.3333, 0.2941, 0.7059, 0.1176, 0.4706, 0.549, 0.3137, 0.4118, 0.451, 0.3725, 0.2549, 0.5098, 0.3529, 0.4118], [0.8627, 0.2549, 0.4118, 0.5294, 0.4706, 0.2549, 0.1961, 0.6275, 0.3922,